In [9]:
import pickle
import re
from statistics import mean
import pandas as pd
from eval_util import compute_footprint_fitness, compute_footprint_matrix, compute_footprint_matrix_pairs, generate_traces_from_tree

task ="pt"
model= "llama"

runs = [0, 1, 2]

result_records = []
individual_results = []

for run in runs:
    try:
        with open(f'eval/generation-ft-results/{task}_{model}_test_activity_{run}.pkl', 'rb') as f:
            data = pickle.load(f)
            print(len(data["labels"]))
            fitness = []
            idx = 0
            for labels, preds in zip(data['labels'], data['preds']):
                idx += 1
                # Output the results
                print("----------------")
                print(labels)
                print("----------")
                print(preds)
                print("----------------")
                
                if task == "dfg":
                    unique_activities = set()
                    
                    preds = preds.lower()
                    labels = labels.lower()
                    # split labels by lnew ine and then turn each line into a pair of activities by splitting on ' -> '
                    label_pairs = []
                    for line in labels.split('\n'):
                        label_pairs.append((line.split('->')[0].strip(), line.split('->')[1].strip())) 
                    unique_activities.update([pair[0] for pair in label_pairs])
                    unique_activities.update([pair[1] for pair in label_pairs])
                    # split preds by lnew ine and then turn each line into a pair of activities by splitting on ' -> '
                    pred_pairs = []
                    for line in preds.split('\n'):
                        try:
                            pred_pairs.append((line.split('->')[0].strip(), line.split('->')[1].strip()))
                        except:
                            print(line)
                    
                    # make sure activities are the same
                    true_matrix = compute_footprint_matrix_pairs(label_pairs, unique_activities)
                    pred_matrix = compute_footprint_matrix_pairs(pred_pairs, unique_activities)
                elif task == "pt":
                    if "*" in labels:
                        print("loop in gt")
                        continue
                    if "*" in preds:
                        print("loop in pred")
                        continue

                    unique_activities = set()
                    # Regex pattern to match strings within single quotes
                    pattern = r"'(.*?)'"

                    # Find all matches
                    matches = re.findall(pattern, labels)
                    unique_activities.update(matches)
                    preds = preds.lower()
                    labels = labels.lower()
                    unique_activities = {act.lower() for act in unique_activities}
                    
                    true_str_traces = generate_traces_from_tree(labels, unique_activities)
                    true_matrix = compute_footprint_matrix(true_str_traces, unique_activities)
                    str_traces = generate_traces_from_tree(preds, unique_activities)
                    pred_matrix = compute_footprint_matrix(str_traces, unique_activities)
                current_fitness = compute_footprint_fitness(true_matrix, pred_matrix)
                print(current_fitness)
                fitness.append(current_fitness)
                individual_results.append({
                    "sample_size": 'max',
                    "run": run,
                    "fitness": current_fitness,
                    "true": labels,
                    "pred": preds,
                    "unique_activities": unique_activities
                })
            rec = {
                "Task": task,
                "Approach": model,
                "sample_size": 'max',
                "run": run,
                "avg_fitness": mean(fitness),
            }
            print(len(fitness))
            print(fitness)
        result_records.append(rec)
        print(result_records)
    except Exception as e:
        print(e)
        continue


1528
----------------
->( 'Navigate to Goods Receipt screen', 'Enter Goods movement info.', 'Select Adopt+ Details"', 'Select Delivery completed flag', 'Save Goods receipt' )
----------
->( 'Navigate to Goods Receipt screen', 'Enter Goods movement info.', 'Select Adopt+ Details"', 'Select Delivery Completed flag', 'Save Goods Receipt' )
----------------
1.0
----------------
->( 'Invalid passport or visa application processed by an agent', 'TA can no longer guarantee the trip', 'Inform the client' )
----------
->( X( 'Invalid passport or visa application processed by an agent', 'Invalid passport or visa application processed by an agent', 'Invalid passport or visa application processed by an agent' ), 'Ta can no longer guarantee the trip', 'Inform the client' )
----------------
Activity not found in the list: skipping x or ta can no longer guarantee the trip
0.7777777777777778
----------------
->( 'Measure customer satisfaction with warranty handling and resolution', 'Monitor and report

In [10]:
# individual results df
df = pd.DataFrame(individual_results)

# add column for size of 'unique_activities'
df['unique_activities_size'] = df['unique_activities'].apply(lambda x: len(x))
# keep only the rows where the size of the unique activities is greater than 1
#df = df[df['unique_activities_size'] > 1]
# group activity len sizes by adding a column for the size ranges of the unique activities in steps of 5
df['unique_activities_size_range'] = df['unique_activities_size'].apply(lambda x: (x//5)*5)
df


,sample_size,run,fitness,true,pred,unique_activities,unique_activities_size,unique_activities_size_range
0,max,0,1.000000,"->( 'navigate to goods receipt screen', 'enter...","->( 'navigate to goods receipt screen', 'enter...","{navigate to goods receipt screen, select adop...",5,5
1,max,0,0.777778,->( 'invalid passport or visa application proc...,->( x( 'invalid passport or visa application p...,{invalid passport or visa application processe...,3,0
2,max,0,0.760000,->( 'measure customer satisfaction with warran...,->( 'monitor and report on warranty management...,"{identify improvement opportunities, measure c...",5,5
3,max,0,0.777778,->( 'user navigates to create your first app` ...,"->( 'routes to register new app` page', 'user ...","{presses `create` button, fills up fields: app...",6,5
4,max,0,0.760000,"->( 'search for book', 'enter book number', 'r...","->( 'enter book number', 'search for book', 'r...","{confirm reservation, enter book number, reser...",5,5
...,...,...,...,...,...,...,...,...
4487,max,2,0.440000,"->( 'halaman my course', 'archive', 'halaman l...","->( 'halaman my course', 'halaman list course ...","{klik course inished, archive, halaman my cour...",5,5
4488,max,2,0.802469,+( ->( 'define process on data maintenance (ad...,"->( 'clean data from ci databases (adi-', 'dep...","{backup bs with new data, deploy environment w...",9,5
4489,max,2,1.000000,"->( 'weight mannualy', 'fill in weighing map f...","->( 'weight mannualy', 'fill in weighing map f...","{pay to angler, charge buyers, place batch in ...",7,5
4490,max,2,0.673469,"->( 'previous activities', 'discard applicatio...","->( 'previous activities', 'convert gpa scores...","{convert gpa scores, assign scholarship, furth...",7,5


In [11]:
import pandas as pd

sap_categorization = pd.read_csv("../data/" + "categorized_processes_by_industry.csv")
apqc_categorization = pd.read_csv("../data/" + "categorized_processes.csv")
# rename 'category' column to 'industry' for sap_categorization
sap_categorization = sap_categorization.rename(columns={"category": "Industry"})
# rename 'category' column to 'Process type' for apqc_categorization
apqc_categorization = apqc_categorization.rename(
    columns={"category": "Process type"}
)

sap_categorization["unique_activities"] = sap_categorization["unique_activities"].apply(lambda x: tuple(sorted([s.lower() for s in eval(x)])))
apqc_categorization["unique_activities"] = apqc_categorization["unique_activities"].apply(lambda x: tuple(sorted([s.lower() for s in eval(x)])))
sap_categorization

,model_id,model_name,unique_activities,Industry
0,e8990a10f516495e89a1eabf9627b9e9,Employee Onboarding,"(add personal data, procure work equipment, se...",Other
1,e89a9255406a491cab3bd827a043dd28,Checkout Tianen Jane Doe,"(accept cash, bag items, process credit card, ...",Retail
2,e89e0c0e06c440e2a006c32d1d25e3f7,Pay,"(fill in bank transfer info, fill in credit ca...",Banking
3,e8a0fe270519408f9df49651977afc8e,Exercise 2,"(statement from the witness, statements received)",Other
4,e8a31cb70a6546b187326a111db648fe,Handle PDF,"(download document, send documents by post, si...",Other
...,...,...,...,...
15897,1df4da306db045a096218b856ab871f5,verification final,"(mark application as ready, perform academic v...",Higher education and research
15898,1e05a6468dc54035a1bc56cf17c754a0,Ex 1.2 - week 5,"(build travel plan, calculate car rental cost,...",Travel and transportation
15899,1e05dda50a7a486dae44a03c47bc5ca4,Example 11,"(customer follow up, risk assessment, simple r...",Other
15900,1e07277a2ff0486cba1cd100be7b4a59,XOR Gateway,"(passenger fails security check, passenger get...",Travel and transportation


In [12]:
# transform "unique_activities" column to a list of strings
df['unique_activities'] = df['unique_activities'].apply(lambda x: tuple(sorted(x)))
df_grouped = df.groupby('unique_activities').agg({'fitness': 'mean'}).reset_index()
df_grouped

,unique_activities,fitness
0,"(, asfwef, start)",1.000000
1,"(, book trip, choose destination, pack suit case)",1.000000
2,"((24) (cp-01) documentation control, (ds- data...",0.604938
3,"(-step de-icing, add information about procedu...",0.750000
4,"(..., backup server databse, pack ludggade, st...",0.791667
...,...,...
1469,"(task, “cso gathers requirement and fills rent...",0.500000
1470,"(update dutch tablet version, update french ta...",1.000000
1471,"(view assignments and reports, view student re...",0.500000
1472,"(view id or passport expiry date, view uploade...",1.000000


In [13]:
# merge with sap_categorization
df_grouped = df_grouped.merge(sap_categorization, on='unique_activities', how='inner')
# merge with apqc_categorization
df_grouped = df_grouped.merge(apqc_categorization, on='unique_activities', how='inner')
# remove duplicates
df_grouped = df_grouped.drop_duplicates(subset=['unique_activities'])

In [14]:
# group by industry and compute the mean fitness and compute the size of each group
df_grouped_industry = df_grouped.groupby('Industry').agg({'fitness': 'mean', 'model_id_x': 'count'}).reset_index()
df_grouped_industry["model_id_x"] = (df_grouped_industry["model_id_x"] / df_grouped_industry["model_id_x"].sum()) *100
# sort by fitness
df_grouped_industry = df_grouped_industry.sort_values(by='fitness', ascending=False)



print(df_grouped_industry.to_latex(
    index=False,
    float_format="%.2f",
    column_format="lcccc",
    escape=False,
))

\begin{tabular}{lcccc}
\toprule
Industry & fitness & model_id_x \\
\midrule
Telecommunications & 1.00 & 0.17 \\
Insurance & 0.93 & 6.96 \\
Procurement & 0.91 & 0.42 \\
Media, sports, and entertainment & 0.90 & 1.01 \\
Retail & 0.88 & 9.22 \\
Logistics/Wholesale distribution & 0.88 & 2.18 \\
Mining & 0.85 & 0.08 \\
Travel and transportation & 0.84 & 5.03 \\
Automotive & 0.83 & 2.60 \\
Banking & 0.83 & 9.05 \\
Utilities & 0.81 & 0.34 \\
Professional services & 0.80 & 2.10 \\
Other & 0.80 & 37.55 \\
Life sciences and healthcare & 0.80 & 9.14 \\
Industrial manufacturing & 0.80 & 3.77 \\
Higher education and research & 0.80 & 5.45 \\
Aerospace and defense & 0.79 & 0.34 \\
High tech & 0.78 & 0.67 \\
Government & 0.78 & 1.01 \\
Consumer products & 0.76 & 1.51 \\
Agribusiness & 0.76 & 0.42 \\
Oil, gas, and energy & 0.70 & 0.08 \\
Defense and security & 0.65 & 0.17 \\
Construction and real estate & 0.61 & 0.75 \\
\bottomrule
\end{tabular}



In [15]:
# group by industry and compute the mean fitness
df_grouped_type = df_grouped.groupby('Process type').agg({'fitness': 'mean', 'model_id_x': 'count'}).reset_index()
df_grouped_type["model_id_x"] = (df_grouped_type["model_id_x"] / df_grouped_type["model_id_x"].sum()) *100
# sort by fitness
df_grouped_type = df_grouped_type.sort_values(by='fitness', ascending=False)
print(df_grouped_type.to_latex(
    index=False,
    float_format="%.2f",
    column_format="lcccc",
    escape=False,
))

\begin{tabular}{lcccc}
\toprule
Process type & fitness & model_id_x \\
\midrule
Manage Human Capital & 0.99 & 0.08 \\
Manage Enterprise Risk, Compliance, Remediation, and Resiliency & 0.88 & 3.77 \\
Manage External Relationships & 0.85 & 6.29 \\
Deliver Physical Products & 0.84 & 16.76 \\
Deliver Services & 0.84 & 8.13 \\
Manage Customer Service & 0.84 & 8.97 \\
Market and Sell Products and Services & 0.83 & 2.60 \\
Other & 0.81 & 27.75 \\
Manage Financial Resources & 0.81 & 10.48 \\
Manage Information Technology & 0.80 & 4.02 \\
Develop and Manage Business Capabilities & 0.78 & 1.76 \\
Acquire, Construct, and Manage Assets & 0.78 & 1.76 \\
Develop and Manage Human Capital & 0.77 & 5.45 \\
Develop and Manage Products and Services & 0.74 & 1.84 \\
Develop Vision and Strategy & 0.68 & 0.34 \\
\bottomrule
\end{tabular}



In [ ]:
# Grouping by multiple categories and computing averages and variance
grouped = df.groupby(['Task', 'Approach']).agg({
    'avg_fitness': ['mean', 'std']
})
# Rounding variance columns to three decimal places
grouped[('avg_fitness', 'std')] = grouped[('avg_fitness', 'std')].round(3)
grouped[('avg_fitness', 'mean')] = grouped[('avg_fitness', 'mean')].round(2)